In [1]:
from pathlib import Path
from skimage import filters
from skimage import io
from skimage import color
from skimage import feature
from skimage import morphology
from skimage import transform
from skimage import draw
from skimage import util
from skimage import measure
from skimage import exposure

import cv2
import re
import numpy as np


from collections import Counter
%matplotlib qt

In [2]:
# V1
import matplotlib
import matplotlib.pyplot as plt
import numpy as np

from skimage import data
from skimage.filters import threshold_multiotsu


source_path = Path('../data/partial_results/wyciete_indeksy')

save_path = Path('../data/partial_results/wyciete_indeksy_wykryte_cyfry')
save_path.mkdir(parents=True, exist_ok=True)

images_directories = source_path.glob("*")
cnt = 0
cnt_good = 0
detections = []
for directory in images_directories:
    images_paths = directory.glob("*.png")
    number_of_image = re.search('[0-9]+', directory.stem)[0]
    image_directory = save_path / number_of_image
    image_directory.mkdir(parents=True, exist_ok=True)
    
    for word_image_path in list(images_paths):
#         word_image_path = Path('../data/partial_results/wyciete_indeksy/3/3.png')
        number_of_word = re.search('[0-9]+', word_image_path.stem)[0]
        cnt += 1
        
        word_directory = image_directory / str(number_of_word)
        word_directory.mkdir(parents=True, exist_ok=True)
    
        print(word_image_path)
        word_image_org = io.imread(word_image_path)
        word_image = word_image_org.copy()
        word_image = color.rgb2gray(word_image)
        

        word_image = filters.gaussian(word_image)
        # Contrast stretching
        p2, p98 = np.percentile(word_image, (2, 98))
        word_image = exposure.rescale_intensity(word_image, in_range=(p2, p98))

        # Applying multi-Otsu threshold for the default value, generating
        # three classes. 
        thresholds = threshold_multiotsu(word_image, classes=3)

        # Using the threshold values, we generate the three regions.
        regions = np.digitize(word_image, bins=thresholds)

        # Jeden z wykrytych regionów odpowiadał w większości jasności kratki, więc go usuwam.
        # Region trzeba traktować jako jakiś przedział wartości jasności w obrazie.
        img2 = word_image*util.invert((regions == 1))
        
        # Po ponownym wykryciu regionów, jeden z nich pasował do cyfr więc użyłem go licząc, że są to cyfry.
        thresholds = threshold_multiotsu(img2, classes=3)
        regions = np.digitize(word_image, bins=thresholds)
        img3 = (regions==0)
        img3 = morphology.dilation(img3, morphology.disk(1)) # DODANE

        # Tutaj region to już chodzi o podobne jasności pikseli w sąsiedztwie.
        label_image = measure.label(img3)
        regions = measure.regionprops(label_image)
        print(len(regions))
        
        #### SPRAWDZIC ###
#         while len(regions) < 6:
#             img3 = morphology.erosion(img3, morphology.disk(1))
#             label_image = measure.label(img3)
#             regions = measure.regionprops(label_image)

        
        counter = 1
        while len(regions) > 6:
            regions = [reg for reg in regions if reg.area > counter]
            counter += 1
        #######################################################################
#         region_mean_area = np.mean([reg.area for reg in regions])
#         region_std_area = np.std([reg.area for reg in regions])
#         regions = [reg for reg in regions if reg.area > region_mean_area/2]
        #######################################################################
#         region_mean_area = np.mean([reg.area for reg in regions])
#         print("region_mean_area: ", region_mean_area)
#         if len(regions) > 7:
#             region_mean_area = np.mean([reg.area for reg in regions])
#             regions = [reg for reg in regions if reg.area > region_mean_area]
            
#         counter = 1
#         while len(regions) < 6:
#             counter += 1
#             regions = [reg for reg in regions if reg.area > (region_mean_area-counter)]
#             if region_mean_area-counter < region_mean_area/2:
#                 break
#         regions = regions_above_mean.copy()    
        #######################################################################
        print("liczba wykrytych regionów (cyfr): ", len(regions))
        
        detections.append(len(regions))
        if len(regions) == 6 :
            cnt_good += 1

        def euclidean_distance(p1, p2):
            return pow(pow(p1[0]-p2[0],2)+pow(p1[1]-p2[1],2),0.5)
        
        def func(region):
            return euclidean_distance((0,0), region.centroid)

        regions = sorted(list(regions), key=func)        
            
        
        word_image = color.gray2rgb(word_image)
#         word_image_org = color.rgb2gray(word_image)
        
        img3 = util.img_as_ubyte(img3)
        temp_image = word_image_org.copy()
        for index_digit, region in enumerate(regions):
            height_min = np.min(region.coords[:, 0])
            width_min = np.min(region.coords[:, 1])
            height_max = np.max(region.coords[:, 0])
            width_max = np.max(region.coords[:, 1])
            
            if height_max > word_image_org.shape[0] or width_max > word_image_org.shape[1]:
                break # TODO niepotrzebne?
            
            rr, cc = draw.rectangle_perimeter((height_min, width_min), (height_max, width_max), shape=word_image_org.shape)         
            temp_image[rr, cc] = (255,0+index_digit*30,0+index_digit*30)
            
#             print(temp_image.shape)
#             print(height_min, width_min)
#             print(height_max, width_max)
            for coord in region.coords:
                height = coord[0]
                width = coord[1]
                temp_image[(height,width)] = (255,0,0)
    
            one_digit =  img3[:, width_min:width_max+1]
            one_digit = transform.resize(one_digit, (28,28))
            io.imsave(arr=one_digit, fname=word_directory / '{}.png'.format(index_digit))
            
        io.imsave(arr=temp_image, fname=word_directory / 'index.png')
#         io.imshow(temp_image)
#         break
#     break



#             for coord in region.coords:
#                 word_image_org[coord[0], coord[1]] = (255,0,0)
        
#         word_image_org = color.rgb2gray(word_image_org) NWM CZY POTRZEBNE       
#         io.imsave(arr=word_image_org, fname=image_directory / '{}.png'.format(image_path.stem))
        
print("{}/{} == {}".format(cnt_good, cnt, (cnt_good/cnt)))
print(Counter(detections))




..\data\partial_results\wyciete_indeksy\1\0.png


Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.


7
liczba wykrytych regionów (cyfr):  6
..\data\partial_results\wyciete_indeksy\1\1.png
18

Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0


liczba wykrytych regionów (cyfr):  6
..\data\partial_results\wyciete_indeksy\1\10.png
13
liczba wykrytych regionów (cyfr):  6
..\data\partial_results\wyciete_indeksy\1\11.png

...

..\data\partial_results\wyciete_indeksy\9\3.png
6
liczba wykrytych regionów (cyfr):  6
..\data\partial_results\wyciete_indeksy\9\4.png
8
liczba wykrytych regionów (cyfr):  6
..\data\partial_results\wyciete_indeksy\9\5.png
9
liczba wykrytych regionów (cyfr):  6


Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0

..\data\partial_results\wyciete_indeksy\9\6.png
1
liczba wykrytych regionów (cyfr):  1
..\data\partial_results\wyciete_indeksy\9\7.png
8
liczba wykrytych regionów (cyfr):  6
..\data\partial_results\wyciete_indeksy\9\8.png
16
liczba wykrytych regionów (cyfr):  6


Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.


..\data\partial_results\wyciete_indeksy\9\9.png
6
liczba wykrytych regionów (cyfr):  6
353/444 == 0.795045045045045
Counter({6: 353, 1: 30, 5: 18, 3: 17, 2: 16, 4: 10})


In [3]:
# zmienione v2
import cv2
source_path = Path('../data/partial_results/wyciete_indeksy')

save_path = Path('../data/partial_results/wyciete_indeksy_zmienione_v2')
save_path.mkdir(parents=True, exist_ok=True)

images_directories = source_path.glob("*")
for directory in images_directories:
    images_paths = directory.glob("*.png")
    number_of_image = re.search('[0-9]+', directory.stem)[0]
    save_directory = save_path / number_of_image
    save_directory.mkdir(parents=True, exist_ok=True)
    
    for image_path in list(images_paths):     
#         number_of_word = re.search('[0-9]+', image_path.stem)[0]
        print(image_path)
        img = io.imread(image_path)
        img = color.rgb2gray(img)
        
        # img = filters.gaussian(img)
        # Contrast stretching
        p2, p98 = np.percentile(img, (2, 98))
        img = exposure.rescale_intensity(img, in_range=(p2, p98))

        # thresh = filters.threshold_otsu(img, 3)
        # img = img > thresh
        # img = morphology.opening(img, morphology.disk(1))


        img = util.img_as_ubyte(img)
        img = cv2.Canny(image=img, threshold1=800, threshold2=1000)

        img = morphology.closing(img, morphology.disk(2))


        label_img = measure.label(img)
        regions = measure.regionprops(label_img)

        def func(region):
            return region.area

        img = color.gray2rgb(img)
        regions = sorted(regions, key=func, reverse=True)
        print(len(regions))
        for index_digit, region in enumerate(regions[:6]):
            height_min = np.min(region.coords[:, 0])
            width_min = np.min(region.coords[:, 1])
            height_max = np.max(region.coords[:, 0])
            width_max = np.max(region.coords[:, 1])
            rr, cc = draw.rectangle_perimeter((height_min, width_min), (height_max, width_max), shape=img.shape)         
            img[rr, cc] = (255,0,0)
        
        io.imsave(arr=img, fname=save_directory / "{}.png".format(number_of_word) )
        

# io.imshow(img)


..\data\partial_results\wyciete_indeksy\1\0.png
9
..\data\partial_results\wyciete_indeksy\1\1.png
19
..\data\partial_results\wyciete_indeksy\1\10.png
6
..\data\partial_results\wyciete_indeksy\1\11.png
18
..\data\partial_results\wyciete_indeksy\1\12.png
7
..\data\partial_results\wyciete_indeksy\1\2.png
7
..\data\partial_results\wyciete_indeksy\1\3.png
6
..\data\partial_results\wyciete_indeksy\1\4.png
8
..\data\partial_results\wyciete_indeksy\1\5.png
9
..\data\partial_results\wyciete_indeksy\1\6.png
5
..\data\partial_results\wyciete_indeksy\1\7.png
6
..\data\partial_results\wyciete_indeksy\1\8.png
7
..\data\partial_results\wyciete_indeksy\1\9.png
5
..\data\partial_results\wyciete_indeksy\10\0.png
5
..\data\partial_results\wyciete_indeksy\10\1.png
14
..\data\partial_results\wyciete_indeksy\10\10.png
6
..\data\partial_results\wyciete_indeksy\10\2.png
8
..\data\partial_results\wyciete_indeksy\10\3.png
8
..\data\partial_results\wyciete_indeksy\10\4.png
8
..\data\partial_results\wyciete_indek

..\data\partial_results\wyciete_indeksy\19\15.png
6
..\data\partial_results\wyciete_indeksy\19\16.png
9
..\data\partial_results\wyciete_indeksy\19\17.png
4
..\data\partial_results\wyciete_indeksy\19\18.png
3
..\data\partial_results\wyciete_indeksy\19\19.png
4
..\data\partial_results\wyciete_indeksy\19\2.png
6
..\data\partial_results\wyciete_indeksy\19\20.png
8
..\data\partial_results\wyciete_indeksy\19\21.png
8
..\data\partial_results\wyciete_indeksy\19\22.png
6
..\data\partial_results\wyciete_indeksy\19\3.png
3
..\data\partial_results\wyciete_indeksy\19\4.png
8
..\data\partial_results\wyciete_indeksy\19\5.png
7
..\data\partial_results\wyciete_indeksy\19\6.png
9
..\data\partial_results\wyciete_indeksy\19\7.png
8
..\data\partial_results\wyciete_indeksy\19\8.png
6
..\data\partial_results\wyciete_indeksy\19\9.png
9
..\data\partial_results\wyciete_indeksy\2\0.png
15
..\data\partial_results\wyciete_indeksy\2\1.png
13
..\data\partial_results\wyciete_indeksy\2\10.png
7
..\data\partial_results

1
..\data\partial_results\wyciete_indeksy\28\13.png
1
..\data\partial_results\wyciete_indeksy\28\14.png
1
..\data\partial_results\wyciete_indeksy\28\15.png
1
..\data\partial_results\wyciete_indeksy\28\2.png
1
..\data\partial_results\wyciete_indeksy\28\3.png
1
..\data\partial_results\wyciete_indeksy\28\4.png
8
..\data\partial_results\wyciete_indeksy\28\5.png
2
..\data\partial_results\wyciete_indeksy\28\6.png
4
..\data\partial_results\wyciete_indeksy\28\7.png
2
..\data\partial_results\wyciete_indeksy\28\8.png
1
..\data\partial_results\wyciete_indeksy\28\9.png
1
..\data\partial_results\wyciete_indeksy\3\0.png
3
..\data\partial_results\wyciete_indeksy\3\1.png
5
..\data\partial_results\wyciete_indeksy\3\10.png
5
..\data\partial_results\wyciete_indeksy\3\11.png
6
..\data\partial_results\wyciete_indeksy\3\12.png
10
..\data\partial_results\wyciete_indeksy\3\13.png
18
..\data\partial_results\wyciete_indeksy\3\14.png
6
..\data\partial_results\wyciete_indeksy\3\2.png
7
..\data\partial_results\wyc

In [54]:
# V3
import matplotlib
import matplotlib.pyplot as plt
import numpy as np

from skimage import data
from skimage.filters import threshold_multiotsu


source_path = Path('../data/partial_results/wyciete_indeksy')

save_path = Path('../data/partial_results/wyciete_indeksy_zmienione_v3')
save_path.mkdir(parents=True, exist_ok=True)

images_directories = source_path.glob("*")
cnt = 0
cnt_good = 0
detections = []
for directory in images_directories:
    images_paths = directory.glob("*.png")
    number_of_image = re.search('[0-9]+', directory.stem)[0]
    image_directory = save_path / number_of_image
    image_directory.mkdir(parents=True, exist_ok=True)
    
    for word_image_path in list(images_paths)[11:]:
        word_image_path = Path('../data/partial_results/wyciete_indeksy/3/3.png')
#         number_of_word = re.search('[0-9]+', word_image_path.stem)[0]
        cnt += 1
        
#         word_directory = image_directory / str(number_of_word)
#         word_directory.mkdir(parents=True, exist_ok=True)
    
        print(word_image_path)
        
        word_image_org = io.imread(word_image_path)
        word_image = word_image_org.copy()
        word_image = color.rgb2gray(word_image)
        

        word_image = filters.gaussian(word_image)
        # Contrast stretching
        p2, p98 = np.percentile(word_image, (2, 98))
        word_image = exposure.rescale_intensity(word_image, in_range=(p2, p98))
        

        # Applying multi-Otsu threshold for the default value, generating
        # three classes. 
        thresholds = threshold_multiotsu(word_image, classes=3)

        # Using the threshold values, we generate the three regions.
        regions = np.digitize(word_image, bins=thresholds)
        

        # Jeden z wykrytych regionów odpowiadał w większości jasności kratki, więc go usuwam.
        # Region trzeba traktować jako jakiś przedział wartości jasności w obrazie.
        img2 = word_image*util.invert((regions == 1))
        
        # Po ponownym wykryciu regionów, jeden z nich pasował do cyfr więc użyłem go licząc, że są to cyfry.
        thresholds = threshold_multiotsu(img2, classes=3)
        regions = np.digitize(word_image, bins=thresholds)
        img3 = (regions==0)
        
        img3 = util.img_as_ubyte(img3)
        img3 = morphology.dilation(img3, morphology.disk(1))
        io.imshow(img3)
        break
        io.imsave(arr=img3, fname=image_directory / "{}.png".format(number_of_word) )
    break

..\data\partial_results\wyciete_indeksy\3\3.png


In [13]:
# import cv2
# source_path = Path('../data/partial_results/wyciete_indeksy')

# save_path = Path('../data/partial_results/wyciete_indeksy_zmienione')
# save_path.mkdir(parents=True, exist_ok=True)

# images_directories = source_path.glob("*")
# for directory in images_directories:
#     images_paths = directory.glob("*.png")
#     number_of_image = re.search('[0-9]+', directory.stem)[0]
#     save_directory = save_path / number_of_image
#     save_directory.mkdir(parents=True, exist_ok=True)
    
#     for image_path in list(images_paths):        
#         print(image_path)
#         img = io.imread(image_path)
#         img = color.rgb2gray(img)

#         img = filters.gaussian(img)
#         # Contrast stretching
#         p2, p98 = np.percentile(img, (2, 98))
#         img = exposure.rescale_intensity(img, in_range=(p2, p98))
        
#         img = util.img_as_ubyte(img)
#         img = cv2.Canny(image=img, threshold1=300, threshold2=400)
        
# #         img = morphology.dilation(img, morphology.disk(3))
#         io.imshow(img)
        
        
#         break
#     break

..\data\partial_results\wyciete_indeksy\1\0.png


Jakie problemy? przykład 3_5 - obraz 3, indeks 5
- dzieli liczby jak 2 czy 3 w połowie na dwie części góra i dół - DONE prawie - jeżeli podzieli liczbę na pół i te dwie połowy wykryje jako osobne regiony to mamy dwa razy 2 (bo wycinamy całą wysokość obrazu zawsze) - przykład 3_5
- cyfry zbyt blisko siebie, aż się prawie dotykają np. zdjęcie 1, ostatni wiersz cyfry 78
- czasami jasność kratki i cyfr jest tak bliska siebie, że cyfry łączą się poprzez kratkę i wykrywa dwie cyfry jako jeden region, np. zdjęcie 1 wiersz 11 (82), zdjęcie 2 wiersz 8 (23)
- trzeba coś wymyślić, zby wyeliminowac wykrywanie zbyt małych regionów (które nie są liczbami)
- wycinanie indeksów - może przydałoby się dodać lekki margines na około
- jak jest bardzo cienka cyfra, np. jedynka w kształcie pojedynczej kreski to po przeskalowaniu wychodzi coś dziwnego, przykład 9_5
- 10_7 - dziwny przypadek, jakoś wymyślić żeby regiony mogły być tylko obok siebie
- przy wycinaniu cyfr też dodać lekki margines
- potrafi 4 podzielić na 2 części lewo-prawo 16_0
- no problem jak 7 cyfr 24_7
- może coś zrobić, aby regiony stykały się między sobą, region  jednej cyfry styka się z regionem drugiej cyfry